# Neural Machine Translation using a Transformer model

Finetune [T5](https://huggingface.co/t5-small) on the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset to translate English text to French.


## Install required libraries

In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install sacrebleu
!pip install --upgrade --no-cache-dir gdown


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6


In [1]:
import locale
# locale.getpreferredencoding = lambda: "UTF-8"

## Imports

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
from sklearn.model_selection import train_test_split
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import pipeline
import pandas as pd
from datasets import load_dataset, load_metric, Dataset

C:\Users\ahmad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Download the dataset
Loading the English-French subset of the [OPUS Books](https://huggingface.co/datasets/opus_books) dataset:

In [3]:
! pip install kaggle

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets download ahmadsameh7810/aic-datasets

 39% 5.00M/12.9M [00:00<00:00, 22.2MB/s]
100% 12.9M/12.9M [00:00<00:00, 46.4MB/s]


In [8]:
! unzip aic-datasets

Archive:  aic-datasets.zip
  inflating: test.jsonl              
  inflating: train.jsonl             
  inflating: validation.jsonl        


## Read Dataset

In [3]:
train = pd.read_json('train.jsonl', lines=True)
validation = pd.read_json('validation.jsonl', lines=True)
test = pd.read_json('test.jsonl', lines=True)


In [4]:
test

,summary,predicted
0,تنظر محكمة عسكرية أمريكية في وقت لاحق من اليوم...,وسيواجه حسن عددا من ضحاياه في قاعة المحكمة لأن...
1,كشفت ألمانيا النقاب عن خطط لإضافة الجزائر والم...,ألمانيا تواجه مصاعب في التعامل مع الأعداد المت...
2,قال التليفزيون السوري إن قوات الحكومة استعادت ...,وكان الجيش السوري قد استعاد قبل نحو شهرين السي...
3,توج فريق الارسنال ببطولة كأس انجلترا لكرة القد...,وفاجأ هال سيتي الحضور بمباغتة الارسنال بهدفين ...
4,يوضع الصراع في العراق غالبا في إطار صراع بين ا...,ويعتقد بأن ما بين 250 ألف إلى 400 ألف من الشبك...
...,...,...
4684,مجموعة مختارة من أفضل الصور في القارة الأفريقي...,رجل سوداني يغادر مخبزا وهو محمل بحقيبة مملوءة ...
4685,استأنف الرئيس الفلسطيني محمود عباس ورئيس الوزر...,"كما شددت على ان بلادها ""ستكون شريكا دائما ونشي..."
4686,هناك الكثير مما يجمع كارول نحاس بزوجها كارلوس ...,الدفاع عن كارلوس بعد هروب كارلوس غصن من إقامته...
4687,افرجت السلطات السودانية عن 57 من معتقلي حركة ا...,دعوة ودعا زعيم الحركة خليل ابراهيم بقية الجماع...


In [24]:
validation

,summary,predicted
0,وقع الحوثيون اتفاقا مع عدد من الاحزاب اليمنية ...,دخل القتال بين القوات الحكومية والمسلحين الحوث...
1,ناقش معلقون في صحف عربية تطور الأزمة السياسية ...,"""معركة"" الحكومة والاتحاد العام للشغل في سياقٍ ..."
2,اتهمت الشرطة البريطانية رسميا رجلا فرنسيا بحيا...,"أعلنت حالة التأهب الأمني بسبب ""تصرفات مشبوهة"" ..."
3,تطلب شابة على إحدى صفحات فيسبوك المغلقة نصيحة ...,"وتتفق عليا خيربك سليمان، مؤسسة مجموعة ""صبايا"" ..."
4,فرضت الولايات المتحدة عقوبات على بنك العملة ال...,قطعت كوريا الشمالية الخط الهاتفي الساخن بين ال...
...,...,...
4684,في الأول من مارس/ آذار الجاري بدأ نحو 27 ألف ج...,استولى تنظيم الدولة الإسلامية على الموصل في يو...
4685,تسود حالة من الهدوء الحذر محيط القصر والشوارع ...,وكانت قوات الأمن، التي كانت متمركزة قريبا من ن...
4686,حذر رئيس حزب العمال الكردستاني السجين، عبدالله...,أدى تقدم مسلحو الدولة الإسلامية تجاه كوباني إل...
4687,تناول بعض الأنواع الشائعة من مسكنات الألم له ع...,وأشار إلى أن هذه المسكنات يشيع استخدامها من جا...


In [5]:
train

,summary,predicted
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,تسيطر قوات روسية حاليا على معظم القواعد العسكر...
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...","وأضاف لبي بي سي ""كان من المفترض أن تتم عملية ت..."
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,الجنة الملعونة: دموع الفقراء في كشمير لماذا يل...
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ومنذ أن حط كوكاي، الذي عرف بعد وفاته باسم كوبو...
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",كما أكد المصدر تمكن مسلحي الحراك من اعتقال مسل...
...,...,...
37514,*هذا الفيديو بدون تعليق*,ويقذف البركان بأنهار من الحمم البركانية عبر ال...
37515,بي بي سي ترند تقرير مي نعمان,لكن القليل من المغردين رأوا ان في هذه الحملة ن...
37516,تداعيات متواصلة,رمضان آخر يستقبله المسلمون في ظل كورونا للعام ...
37517,حملة شامتة,ومع وصول ذاك الجدل، الذي تحدث عنه تسيدال إلى ذ...


In [8]:
train = train[:int(len(train)*0.1)]
validation = validation[:int(len(validation)*0.1)]
test = test[:int(len(test)*0.1)]


In [9]:
train

,summary,predicted
0,بدأت القوات الأوكرانية الانسحاب من شبه جزيرة ا...,تسيطر قوات روسية حاليا على معظم القواعد العسكر...
1,"""هل سيتم تغيير العبارة الشهيرة للمؤرخ اليوناني...","وأضاف لبي بي سي ""كان من المفترض أن تتم عملية ت..."
2,قالت الشرطة في القطاع الهندي من إقليم كشمير إن...,الجنة الملعونة: دموع الفقراء في كشمير لماذا يل...
3,في عام 816، تجول راهب يدعى كوكاي، في المنحدرات...,ومنذ أن حط كوكاي، الذي عرف بعد وفاته باسم كوبو...
4,"أكد مصدر في ""الحراك التهامي"" لأبناء محافظة الح...",كما أكد المصدر تمكن مسلحي الحراك من اعتقال مسل...
...,...,...
3746,في الوقت الذي تتواصل فيه عمليات الجيش اليمني ض...,إلا أن الباحث والخبير المتخصص في شؤون تنظيم ال...
3747,"ربما يكون قرابة نصف معارفنا وأصدقائنا من ""الأص...",في أغلب الأحيان نتجاهل أخطاء الأصدقاء الأعداء،...
3748,"في كل مكان في اليابان تتردد أصداء عبارة ""سومي ...",يقال إن قيم التعاون والبذل من أجل لنفع العام ن...
3749,اعتذرت زوجة وزير الداخلية الاسرائيلي سيلفان شا...,وقالت جودي شالوم، التي تقدم برنامجا اذاعيا حوا...


In [10]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)
validation = Dataset.from_pandas(validation)

### Load the ``T5`` tokenizer to process the English-French language pairs:

In [11]:
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

The preprocessing steps we need to create are:

1. Prefix the input with a prompt so T5 knows this is a translation task. Some models capable of multiple NLP tasks require prompting for specific tasks.

2. Tokenize the input (English) and target (French).


3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [12]:
source_lang = "predicted"
target_lang = "summary"
prefix = "Summarize "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Apply the preprocessing function over the entire dataset:
- use the ``map`` method.
- ``batched=True`` to process multiple elements of the dataset at once.

In [13]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)
tokenized_validation = validation.map(preprocess_function, batched=True)

In [14]:
print(tokenized_train[10])

{'summary': 'يمكن للشعور بالذنب أن يجعل الأشياء التي تثير رغبتنا تبدو أكثر إغراء. فهل يمكن أن نتخلى قليلا عن بعض قواعد الحمية الغذائية التي نتبعها مثلا، ثم يكون في ذلك فائدة صحية ونفسية لنا؟ الصحفي ديفيد روبسون يحاول الإجابة عن هذا السؤال.', 'predicted': 'وتضيف غولدسمث: "أعتقد أن ما قد يُهمل هنا في هذا الشأن أمر واحد: وهو أنه لا بأس من أن ندلل أنفسنا، فمن الجميل أن ينتابنا شعور بالمتعة نتيجة الانغماس فيما يجلب اللذة. والنتيجة هي أن أولئك الذين كانوا قد أكملوا سابقاً كلمات أو جملاً تدل على الخطايا كانوا لاحقاً أكثر ميلا إلى أن يملأوا الفراغات بكلمات مرادفة للرغبات مثل ’تمتع‘ أو ’متعة‘ مقارنة بغيرها من الكلمات الأكثر إيجابية.', 'input_ids': [12198, 1635, 1737, 3, 2, 3, 2, 10, 96, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 10, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 5, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 

### Dynamically pad the inputs received

In [15]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_name)

## Evaluation Metric
Including a metric for evaluating the model's performance. You can quickly load a evaluation method with the [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [SacreBLEU](https://huggingface.co/spaces/evaluate-metric/sacrebleu) metric.

In [16]:
metric = evaluate.load("rouge")

In [17]:
# Model might produce some extra spaces or line breaks
# that are not present in the ground truth labels, and these could affect the evaluation metrics.
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]

    # decode the predicted values into human-readable text
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace all occurrences of -100 in the labels array with the ID of the pad token
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    # decode the labels values into human-readable text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"rougl": result["score"]}

    # round it to four decimal places
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Training

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.

3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

In [23]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    save_total_limit=3,
    num_train_epochs=1,
    # reduce memory usage
    # fp16=True,

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)


In [24]:
trainer.train()

In [ ]:
trainer.save_model('T5_checkpoint')

In [ ]:
!cp -r /content/T5_checkpoint /content/

cp: '/content/T5_checkpoint' and '/content/T5_checkpoint' are the same file


# Testing

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."
text2 = "my name is John"
text3 = 'he died'

In [ ]:
def predict(sentence):
  inputs = tokenizer(sentence, return_tensors="pt").input_ids
  model = AutoModelForSeq2SeqLM.from_pretrained("/content/T5_checkpoint")
  outputs = model.generate(inputs)
  outputs = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return outputs

In [ ]:
pred = predict(text)
pred

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1288: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Les légumes partagent les ressources avec les bactéries fixatrice'

In [ ]:
pred = predict(text2)
pred

'..'